# Data Mining

1. Display the collected data.

2. Deal with data and make it the csv file we want.
   1. 1 second time window


In [1]:
import pandas as pd
from datetime import datetime

In [2]:
cheatData = []
for ad in ['../raw_data/user_1_1.log', '../raw_data/user_2_1.log']:#, '../data/normal_user1_1.log']:
    f = open(ad)
    cheatData+=f.readlines()

## Display the raw data


In [3]:
print(len(cheatData))
cheatData[:5]

17299


['label: 0, timestamp: 1650243539, dist_s: 628.077, angle_s: 230.351, mean_dist: 5764.6, min_dist: 5094.55, view_dist: 74.2955, key_2: 1, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 0\n',
 'label: 0, timestamp: 1650243540, dist_s: 984.68, angle_s: 0, mean_dist: -nan, min_dist: 1e+07, view_dist: 74.2955, key_45: 1, key_121: 2, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 0\n',
 'label: 0, timestamp: 1650243541, dist_s: 1247.55, angle_s: 29.7836, mean_dist: 5807.95, min_dist: 5296.89, view_dist: 64.6675, key_45: 1, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 0\n',
 'label: 0, timestamp: 1650243542, dist_s: 1300.78, angle_s: 28.9494, mean_dist: 5955.68, min_dist: 5187.03, view_dist: 81.906, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 12\n',
 'label: 0, timestamp: 1650243543, dist_s: 1363.45, angle_s: 3.18005, mean_dist: 7257.5, min_dist: 7221.94, view_dist: 80.5923, shooting: 0, aiming: 0, moving: 0, average_mouse_speed: 4\n']

## Data Preprocess


In [4]:
# define some consts
useful_key_set = [87, 65, 83, 68, 81, 69, 162, 9, 70, 71, 49, 50, 51, 52]

In [5]:
# define useful and useless key press function
def useful_key_press(keyname, value, ret):
    keynum = int(keyname[4:])
    presstime = int(value)
    if keynum in useful_key_set:
        ret['useful'] += presstime
    else:
        ret['useless'] += presstime

# for each data define a function to process it, return a dictionary
def convert_line_to_dict(record:str, keymode:str='useful')->dict:
    ret = dict()

    # default value for useful key
    if keymode == 'useful':
        ret['useful'] = 0
        ret['useless'] = 0

    features = record.split(',')
    for f in features:
        [keyname, value] = f.split(': ')
        keyname = keyname[1:] if keyname[0] == ' ' else keyname
        # timestamp
        if keyname=='timestamp':
            ret['timestamp'] = datetime.fromtimestamp(int(value))
            continue
        # average_mouse_speed
        if keyname=='average_mouse_speed':
            ret['average_mouse_speed'] = int(value)
            continue
        # dist_s
        if keyname=='dist_s':
            ret['dist_s'] = float(value)
            continue
        # angle_s
        if keyname =='angle_s':
            ret['angle_s'] = float(value)
            continue
        # mean_dist
        if keyname == 'mean_dist':
            if value == '-nan':
                ret['mean_dist'] = None
            else:
                ret['mean_dist'] = float(value)
            continue
        # min_dist
        if keyname == 'min_dist':
            if value == '1e+07':
                ret['min_dist'] = None
            else:
                ret['min_dist'] = float(value)
            continue
        # view_dist
        if keyname == 'view_dist':
            if value == '-1':
                ret['view_dist'] = None
            else:
                ret['view_dist'] = float(value)
            continue
        # shooting
        if keyname == 'shooting':
            ret['shooting'] = int(value)
            continue
        # aiming
        if keyname == 'aiming':
            ret['aiming'] = int(value)
            continue
        # moving
        if keyname == 'moving':
            ret['moving'] = int(value)
            continue
        # key input
        if keyname[:3]=='key':
            # depends on how you handle the key
        
            # use useful and useless key record
            if keymode=='useful':
                useful_key_press(keyname, value, ret)
                continue
        # label
        if keyname=='label':
            ret['class'] = int(value)
    # if not labeled then its normal data
    # if 'class' not in ret:
    #     ret['class'] = 0
    return ret

def convert_dicts_to_dataframe(dicts, keymode:str='useful'):
    labels = list(dicts[0].keys())
    arrs = [list(d.values()) for d in dicts]
    df = pd.DataFrame(arrs, columns = labels)
    if keymode == 'useful':
        df = df[['timestamp', 'dist_s', 'angle_s', 'mean_dist', \
            'min_dist', 'view_dist', 'average_mouse_speed', \
            'useful', 'useless', 'aiming', 'shooting', 'moving', 'class']]
    return df


In [6]:
dicts = [convert_line_to_dict(record) for record in cheatData]
df = convert_dicts_to_dataframe(dicts)
df

,timestamp,dist_s,angle_s,mean_dist,min_dist,view_dist,average_mouse_speed,useful,useless,aiming,shooting,moving,class
0,2022-04-17 20:58:59,628.077,230.35100,5764.60,5094.55,74.2955,0,0,1,0,0,0,0
1,2022-04-17 20:59:00,984.680,0.00000,NaN,NaN,74.2955,0,0,3,0,0,0,0
2,2022-04-17 20:59:01,1247.550,29.78360,5807.95,5296.89,64.6675,0,0,1,0,0,0,0
3,2022-04-17 20:59:02,1300.780,28.94940,5955.68,5187.03,81.9060,12,0,0,0,0,0,0
4,2022-04-17 20:59:03,1363.450,3.18005,7257.50,7221.94,80.5923,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17294,2022-04-18 02:47:57,1200.150,0.00000,2560.58,2416.79,83.2115,0,0,0,0,0,0,0
17295,2022-04-18 02:47:58,1266.950,0.00000,2746.02,2625.07,83.2115,0,0,0,0,0,0,0
17296,2022-04-18 02:47:59,1221.210,0.00000,2996.24,2868.48,83.2115,0,0,1,0,0,0,0
17297,2022-04-18 02:48:00,1228.330,0.00000,3231.47,3080.77,83.2115,0,0,1,0,0,0,0


In [7]:
df.to_csv('../csv_data/user_1.csv', index=False)